# Scikit-Learn Model Deployments for SVR, RF, Boosting Algorithms

### Importing libraries

In [1]:
import sys
import numpy as np
from Crypto.Cipher import AES
from sklearn.tree import _tree

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Generate Decision-Tree code

In [2]:
def tree_to_code(tree, feature_names, num):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("\n  def tree_{}({}):".format(num, 'X'))
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:".format(indent))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 2)

### Encrypt & Decrypt model

In [3]:
def encrypt_model(folder_path, model_name):
    iv = b'abcdefghijklmnop'
    secret_key = b'abcdefghijklmKEY'
    
    BS = 16
    pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)
    
    filename = folder_path + model_name + '_class'
    ot = open(filename + '.py','rb')
    x = (str(ot.read().decode("utf-8")))
    cipher = AES.new(secret_key,AES.MODE_CBC,iv)
    x = pad(x)
    ENCODED= (cipher.encrypt(x.encode("utf8")))
    ot.close()
    ot = open(filename + '_enc.txt','wb')
    ot.write(ENCODED)
    ot.close()
    
def decrypt_model(folder_path, model_name):
    unpad = lambda s : s[0:-s[-1]]
    filename = folder_path + model_name + '_class'
    
    iv = b'abcdefghijklmnop'
    secret_key = b'abcdefghijklmKEY'
    ot = open(filename + '_enc.txt','rb')
    X_DS = ot.read()
    cipher = AES.new(secret_key,AES.MODE_CBC,iv)
    DECODE = cipher.decrypt(X_DS)
    
    prog_block = unpad(DECODE)
    return prog_block

### Save function: SVR/RF/Boosting model

In [4]:
def save_model(folder_path, model_name, model_save, model_input, model_output, model_type):
    original_stdout = sys.stdout # Save a reference to the original standard output
    model_par_len = model_input.shape[1]
    model_X_mean = list(model_input.mean(axis = 0))
    model_X_std = list(model_input.std(axis = 0))
    model_X_var = ((model_input-model_input.mean(axis = 0))/model_input.std(axis = 0)).var()
    
    py_filename = folder_path + model_name + '_class.py'
    
    if model_type == 'SVR':
        with open(py_filename, 'w') as f:
            sys.stdout = f 
            print("class " + model_name + "():")
            print("  model_type = '" + str(model_type) + "'")
            print("  kernel = '" + str(model_save[1].kernel) + "'")
            print("  degree = " + str(model_save[1].degree))
            print("  gamma = '" + str(model_save[1].gamma) + "'")
            print("  coef0 = " + str(model_save[1].coef0))
            print("  epsilon = " + str(model_save[1].epsilon))
            print("  n_features = " + str(model_par_len))
            print("  input_variance = " + str(model_X_var))
            print("  intercept = " + str(model_save[1].intercept_[0]))
            print("  mean_value = " + str(model_X_mean))
            print("  std_value = " + str(model_X_std))
            print("  dual_coef = " + str(model_save[1].dual_coef_.tolist()))
            print("  support_vectors = " + str(model_save[1].support_vectors_.tolist()))
    elif model_type == 'RF':
        with open(py_filename, 'w') as f:
            sys.stdout = f 
            print("class " + model_name + "():")
            print("  model_type = '" + str(model_type) + "'")
            print("  mean_value = " + str(model_X_mean))
            print("  std_value = " + str(model_X_std))
            [tree_to_code(model_save[1].estimators_[i], ['X['+str(j)+']' for j in range(model_par_len)], i) for i in range(model_save[1].n_estimators)]
    elif model_type == 'GB':
        with open(py_filename, 'w') as f:
            sys.stdout = f 
            print("class " + model_name + "():")
            print("  model_type = '" + str(model_type) + "'")
            print("  learning_rate = " + str(model_save[1].learning_rate))
            print("  base_estimator_value = " + str(model_output.mean()))
            print("  mean_value = " + str(model_X_mean))
            print("  std_value = " + str(model_X_std))
            [tree_to_code(model_save[1].estimators_[i][0], ['X['+str(j)+']' for j in range(model_par_len)], i) for i in range(model_save[1].n_estimators)]
        
    sys.stdout = original_stdout 

### Predict function: SVR/RF/Boosting model

In [5]:
def custom_predict(model, x_input):
    output_list = []
    if model.model_type == 'SVR':
        for sample_i in range(x_input.shape[0]):
            x_input_sample = (x_input[sample_i] - model.mean_value)/model.std_value           
            sup_vecs = np.array(model.support_vectors)
            dual_coefs = np.array(model.dual_coef)
            intercept = model.intercept
            n_featrs = model.n_features
            gamma = (1/n_featrs) if model.gamma == 'auto' else ((1 / (n_featrs * model.input_variance)) if model.gamma == 'scale' else float(model.gamma))
            if model.kernel == 'linear':
                output_sample = (np.dot(dual_coefs, np.dot(sup_vecs, x_input_sample.reshape(1, -1).T)) + model.intercept).flatten()[0]
            elif model.kernel == 'rbf':
                diff = sup_vecs - x_input_sample
                norm2 = np.array([np.linalg.norm(diff[n, :]) for n in range(np.shape(sup_vecs)[0])])
                output_sample = (np.dot(dual_coefs, (np.exp(-gamma*(norm2**2)))) + intercept).flatten()[0]
            elif model.kernel == 'sigmoid':
                output_sample = (np.dot(dual_coefs, (np.tanh(gamma * np.dot(sup_vecs, x_input_sample.reshape(1, -1).T) + model.coef0)))+ intercept).flatten()[0]
            elif model.kernel == 'poly':
                output_sample = (dual_coefs.dot((gamma * np.dot(sup_vecs, x_input_sample.reshape(1, -1).T) + model.coef0)**model.degree)+ intercept).flatten()[0]
            output_list.append(output_sample)
    elif model.model_type == 'RF':
        for sample_i in range(x_input.shape[0]):
            x_input_sample = (x_input[sample_i] - model.mean_value)/model.std_value
            output_sample = np.mean([getattr(model, dir(model)[i])([x for x in x_input_sample]) for i in range(len(dir(model))) if 'tree_' in dir(model)[i]])
            output_list.append(output_sample)
    elif model.model_type == 'GB':
        for sample_i in range(x_input.shape[0]):
            x_input_sample = (x_input[sample_i] - model.mean_value)/model.std_value
            output_sample_x = np.array([getattr(model, dir(model)[i])([x for x in x_input_sample]) for i in range(len(dir(model))) if 'tree_' in dir(model)[i]]).flatten()
            output_sample = sum([model.base_estimator_value] + [model.learning_rate * x for x in output_sample_x])
            output_list.append(output_sample)
    
    return output_list

### Testing

In [6]:
# SVR

n_samples, n_features = 10, 5
rng = np.random.RandomState(0)
y = rng.randn(n_samples)
X = rng.randn(n_samples, n_features)

# Model Training
algo = SVR(C=1, degree=1, epsilon=0.01, kernel='linear')
model_SVR_pl = Pipeline([('standardize',StandardScaler()),('svr',algo)])
model_SVR_sk = model_SVR_pl.fit(X, y)
print('Sklearn prediction: ', model_SVR_sk.predict(X))

# Model Save
folder_path = 'saved_models\\'
model_name = 'svr_model'
encrypt_model(folder_path, model_name)
save_model(folder_path, model_name, model_SVR_sk, X, y,'SVR')

# Model Load
mdl_script = decrypt_model(folder_path, model_name)
svr_model = None
exec(mdl_script)

# Model Prediction
print('Saved model prediction: ', custom_predict(svr_model, X))

Sklearn prediction:  [ 1.17360028  0.38984398  0.98842464  1.92065676  0.93948422  0.00352724
  0.96067962 -0.16132168  0.37597762  0.69535565]
Saved model prediction:  [1.173600276418813, 0.3898439762212129, 0.988424643053091, 1.9206567627425617, 0.9394842191478621, 0.003527237790000526, 0.9606796154228735, -0.16132167916173545, 0.3759776222648109, 0.6953556467227138]


In [7]:
# Random Forest

n_samples, n_features = 10, 5
rng = np.random.RandomState(0)
y = rng.randn(n_samples)
X = rng.randn(n_samples, n_features)

# Model Training
algo = RandomForestRegressor(random_state=0, n_jobs=-1)
model_RF_pl = Pipeline([('standardize',StandardScaler()),('rf',algo)])
model_RF_sk = model_RF_pl.fit(X, y)
print('Sklearn prediction: ', model_RF_sk.predict(X))

# Model Save
folder_path = 'saved_models\\'
model_name = 'rf_model'
encrypt_model(folder_path, model_name)
save_model(folder_path, model_name, model_RF_sk, X, y,'RF')

# Model Load
mdl_script = decrypt_model(folder_path, model_name)
rf_model = None
exec(mdl_script)

# Model Prediction
print('Saved model prediction: ', custom_predict(rf_model, X))

Sklearn prediction:  [ 1.23608284  0.4694954   1.03755037  1.73906065  1.1580774  -0.48150228
  0.98933155  0.28366101  0.26809519  0.45716256]
Saved model prediction:  [1.2360828447, 0.4694953975000001, 1.0375503654, 1.7390606534999995, 1.1580774028999998, -0.48150227570000015, 0.9893315549, 0.2836610051, 0.26809519619999994, 0.45716255619999996]


In [8]:
# Gradient Boosting

n_samples, n_features = 10, 5
rng = np.random.RandomState(0)
y = rng.randn(n_samples)
X = rng.randn(n_samples, n_features)

# Model Training
algo = GradientBoostingRegressor(random_state=0)
model_GB_pl = Pipeline([('standardize',StandardScaler()),('gb',algo)])
model_GB_sk = model_GB_pl.fit(X, y)
print('Sklearn prediction: ', model_GB_sk.predict(X))

# Model Save
folder_path = 'saved_models\\'
model_name = 'gb_model'
encrypt_model(folder_path, model_name)
save_model(folder_path, model_name, model_GB_sk, X, y,'GB')

# Model Load
mdl_script = decrypt_model(folder_path, model_name)
gb_model = None
exec(mdl_script)

# Model Prediction
print('Saved model prediction: ', custom_predict(gb_model, X))

Sklearn prediction:  [ 1.76379795  0.40030092  0.97873617  2.24037407  1.86717872 -0.97680018
  0.95006462 -0.15112746 -0.10279718  0.41050409]
Saved model prediction:  [1.7637979439632852, 0.40030091920377264, 0.9787361664782348, 2.2403740617288346, 1.867178715969322, -0.9768001802711654, 0.9500646182037723, -0.15112746227116527, -0.10279718179622746, 0.4105040812037727]
